# Advent of Code - Day 9: Rope Bridge

## Question 1

This rope bridge creaks as you walk along it. You aren't sure how old it is, or whether it can even support your weight.

It seems to support the Elves just fine, though. The bridge spans a gorge which was carved out by the massive river far below you.

You step carefully; as you do, the ropes stretch and twist. You decide to distract yourself by modeling rope physics; maybe you can even figure out where not to step.

Consider a rope with a knot at each end; these knots mark the head and the tail of the rope. If the head moves far enough away from the tail, the tail is pulled toward the head.

Due to nebulous reasoning involving Planck lengths, you should be able to model the positions of the knots on a two-dimensional grid. Then, by following a hypothetical series of motions (your puzzle input) for the head, you can determine how the tail will move.

Due to the aforementioned Planck lengths, the rope must be quite short; in fact, the head (H) and tail (T) must always be touching (diagonally adjacent and even overlapping both count as touching):

....\
.TH.\
....\

....\
.H..\
..T.\
....\

...\
.H. (H covers T)\
...\
If the head is ever two steps directly up, down, left, or right from the tail, the tail must also move one step in that direction so it remains close enough:

.....    .....    .....\
.TH.. -> .T.H. -> ..TH.\
.....    .....    .....\

...    ...    ...\
.T.    .T.    ...\
.H. -> ... -> .T.\
...    .H.    .H.\
...    ...    ...
Otherwise, if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up:

.....    .....    .....\
.....    ..H..    ..H..\
..H.. -> ..... -> ..T..\
.T...    .T...    .....\
.....    .....    .....\

.....    .....    .....\
.....    .....    .....\
..H.. -> ...H. -> ..TH.\
.T...    .T...    .....\
.....    .....    .....\
You just need to work out where the tail goes as the head follows a series of motions. Assume the head and the tail both start at the same position, overlapping.

For example:

R 4\
U 4\
L 3\
D 1\
R 4\
D 1\
L 5\
R 2\
This series of motions moves the head right four steps, then up four steps, then left three steps, then down one step, and so on. After each step, you'll need to update the position of the tail if the step means the head is no longer adjacent to the tail. Visually, these motions occur as follows (s marks the starting position as a reference point):

== Initial State ==\

......\
......\
......\
......\
H.....  (H covers T, s)

== R 4 ==

......\
......\
......\
......\
TH....  (T covers s)\

......\
......\
......\
......\
sTH...\

......\
......\
......\
......\
s.TH..

......\
......\
......\
......\
s..TH.

== U 4 ==\

......\
......\
......\
....H.\
s..T..

......\
......\
....H.\
....T.\
s.....

......\
....H.\
....T.\
......\
s.....

....H.\
....T.\
......\
......\
s.....

== L 3 ==

...H..\
....T.\
......\
......\
s.....

..HT..\
......\
......\
......\
s.....

.HT...\
......\
......\
......\
s.....

== D 1 ==

..T...\
.H....\
......\
......\
s.....\

== R 4 ==

..T...\
..H...\
......\
......\
s.....

..T...\
...H..\
......\
......\
s.....

......\
...TH.\
......\
......\
s.....

......\
....TH\
......\
......\
s.....

== D 1 ==

......\
....T.\
.....H\
......\
s.....

== L 5 ==

......\
....T.\
....H.\
......\
s.....\

......\
....T.\
...H..\
......\
s.....

......\
......\
..HT..\
......\
s.....

......\
......\
.HT...\
......\
s.....

......\
......\
HT....\
......\
s.....

== R 2 ==

......\
......\
.H....  (H covers T)\
......\
s.....

......\
......\
.TH...\
......\
s.....
After simulating the rope, you can count up all of the positions the tail visited at least once. In this diagram, s again marks the starting position (which the tail also visited) and # marks other positions the tail visited:

..##..\
...##.\
.####.\
....#.\
s###..
So, there are 13 positions the tail visited at least once.

Simulate your complete hypothetical series of motions. How many positions does the tail of the rope visit at least once?

### Discussion 
Need to think about how to handle the size of the grid. Do we know how large it is to begin with? Will need to add to the 2d array as we move more. We may need to expand in all directions, so if we are adding up or left, we will need to reindex. The actual positions in the end do not matter. Only the number of positions seen. 

Using a binary tracker (so 0s for unseen and 1 for seen, will allow for easy summation at the end. 

In [16]:
def tail_path(file_path, num_knots):
    
    assert num_knots >= 2, "Must have at least 2 knots"
    
    # open connection to text file
    motions = open(file_path, 'r')
    
    # create list of current knots locations
    knots = [[0,0]]*num_knots 

    # create list of coordinates last has been 
    last_been = [[0,0]]

    # loop over the lines of the txt file (i.e. the instructins)
    for motion in motions:
        # split instructions to direction and magnitude 
        direction, magnitude = list(motion.strip('\n').split(' '))
        magnitude = int(magnitude)
        
        
        # loop over step in direction
        for m in range(magnitude):
            # create and update new H position 
            new_h = knots[0].copy()
            
            if direction == 'U':
                new_h[0] -= 1
                
            elif direction == 'D':
                new_h[0] += 1
            
            elif direction == 'L':
                new_h[1] -= 1
                
            elif direction == 'R':
                new_h[1] += 1
                

            curr_pre = knots[0].copy()
            knots[0] = new_h.copy()
            
            for knot in range(1, num_knots):
                

                # if updated knot before is in a good place, can break the loop
                if ((knots[knot] == knots[knot-1]) or (abs(knots[knot][0] - knots[knot-1][0]) == 1 
                                          and abs(knots[knot][1] - knots[knot-1][1]) == 1) or (
                    (knots[knot-1][0] == knots[knot][0]) and (abs(knots[knot-1][1] - knots[knot][1]) == 1)) or (
                    (knots[knot-1][1] == knots[knot][1]) and (abs(knots[knot-1][0] - knots[knot][0]) == 1))):
                    pass

                # if T does move, it goes to the old H spot
                else:
                    curr_pre_pre = curr_pre.copy() # unupdated knot from previous iteration
                    curr_pre = knots[knot].copy() # unupdated current knot for next iteration
                    knots[knot] = curr_pre_pre # update knots
  
    
                if (knot == num_knots-1) and (knots[knot] not in last_been):
                        last_been.append(knots[knot])

    # return length of coordinates list   
    return (len(last_been))
                                       
        

In [18]:
tail_path('data/rope.txt', num_knots=2)

6522

## Question 2

A rope snaps! Suddenly, the river is getting a lot closer than you remember. The bridge is still there, but some of the ropes that broke are now whipping toward you as you fall through the air!

The ropes are moving too quickly to grab; you only have a few seconds to choose how to arch your body to avoid being hit. Fortunately, your simulation can be extended to support longer ropes.

Rather than two knots, you now must simulate a rope consisting of ten knots. One knot is still the head of the rope and moves according to the series of motions. Each knot further down the rope follows the knot in front of it using the same rules as before.

Using the same series of motions as the above example, but with the knots marked H, 1, 2, ..., 9, the motions now occur as follows:


Simulate your complete series of motions on a larger rope with ten knots. How many positions does the tail of the rope visit at least once?

In [61]:
def var_tail_path(file_path, num_knots):
    
    assert num_knots >= 2, "Must have at least 2 knots"
    
    # open connection to text file
    motions = open(file_path, 'r')
    
    # create list of current knots locations
    knots = [[0,0]]*num_knots 

    # create list of coordinates last has been 
    last_been = [[0,0]]

    # loop over the lines of the txt file (i.e. the instructins)
    for motion in motions:
        # split instructions to direction and magnitude 
        direction, magnitude = list(motion.strip('\n').split(' '))
        magnitude = int(magnitude)
        
        
        # loop over step in direction
        for m in range(magnitude):
            # create and update new H position 
            new_h = knots[0].copy()
            
            if direction == 'U':
                new_h[0] -= 1
                
            elif direction == 'D':
                new_h[0] += 1
            
            elif direction == 'L':
                new_h[1] -= 1
                
            elif direction == 'R':
                new_h[1] += 1
                

            curr_pre = knots[0].copy()
            knots[0] = new_h.copy()
            
            for knot in range(1, num_knots):
                


                if ((knots[knot] == knots[knot-1]) # the same 
                    or ( # or adjacent 
                    (knots[knot-1][0] == knots[knot][0]) and (abs(knots[knot-1][1] - knots[knot][1]) == 1)) or (
                    (knots[knot-1][1] == knots[knot][1]) and (abs(knots[knot-1][0] - knots[knot][0]) == 1))):
                    break
                
                # if diagonal
                elif (abs(knots[knot][0] - knots[knot-1][0]) == 1 
                      and abs(knots[knot][1] - knots[knot-1][1]) == 1):
                    
                    # if it was previously adjacent
                    if (abs(knots[knot][0] - curr_pre[0]) == 0) or (abs(knots[knot][1] - curr_pre[1]) == 0):
                        knots[knot][0] += knots[knot][0] - curr_pre[0]
                        knots[knot][1] += knots[knot][0] - curr_pre[1]
                    else:
                        break
                        
                else:
                    curr_pre_pre = curr_pre.copy() # unupdated knot from previous iteration
                    curr_pre = knots[knot].copy() # unupdated current knot for next iteration
                    knots[knot] = curr_pre_pre # update knots
  
    
                if (knot == num_knots-1) and (knots[knot] not in last_been):
                        last_been.append(knots[knot])
            print(knots[-1])
    # return length of coordinates list   
    return (len(last_been))
                                       
        

In [64]:
var_tail_path('data/practice_rope.txt', num_knots=10)

[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 1]
[0, 2]
[0, 3]
[0, 5]
[0, 5]
[-1, 5]
[-2, 5]
[-3, 5]
[-4, 5]
[-5, 5]
[-6, 5]
[-8, 5]
[-8, 5]
[-8, 4]
[-8, 3]
[-8, 3]
[-8, 2]
[-8, 1]
[-8, 0]
[-8, -1]
[-8, -3]
[-7, -3]
[-5, -3]
[-5, -2]
[-5, -1]
[-5, 0]
[-5, 1]
[-5, 2]
[-5, 3]
[-5, 4]
[-5, 5]
[-5, 6]
[-5, 6]
[-5, 7]
[-5, 8]
[-5, 9]
[-5, 10]
[-5, 11]
[-5, 12]
[-5, 14]
[-4, 14]
[-3, 14]
[-3, 14]
[-2, 14]
[-1, 14]
[0, 14]
[1, 14]
[2, 14]
[3, 14]
[5, 14]
[5, 13]
[5, 12]
[5, 11]
[5, 10]
[5, 9]
[5, 8]
[5, 7]
[5, 6]
[5, 5]
[5, 4]
[5, 3]
[5, 2]
[5, 1]
[5, 0]
[5, -1]
[5, -2]
[5, -3]
[5, -3]
[5, -4]
[5, -5]
[5, -6]
[5, -7]
[5, -8]
[5, -9]
[5, -11]
[4, -11]
[3, -11]
[2, -11]
[1, -11]
[0, -11]
[-1, -11]
[-2, -11]
[-3, -11]
[-4, -11]
[-5, -11]
[-6, -11]


80